In [1]:
# 필요 패키지 선언언
import pandas as pd
from konlpy.tag import Mecab
from gensim import corpora, models



In [2]:
# 파일 업로드 및 데이터 불러오기
df = pd.read_csv("data/skhynix/leadership_news.csv")
documents = df['content'].dropna().tolist()
documents

['SK하이닉스 경영의 중심에는 ‘SKMS(SK Management System)’가 있다. 구성원들의 행복을 증진하고 이를 통해 VWBE*와 SUPEX*를 추구하는 경영 철학인 SK그룹의 SKMS는 오늘날 SK하이닉스의 발전을 이끌어온 핵심 가치 중 하나다.\nSKMS 실천을 위한 SK하이닉스의 여러 조직 중 구성원들의 성장 및 인적자원개발(이하 HRD, Human Resource Development)을 담당하는 ‘SKMS&Growth’ 조직은 SKMS를 전파하고 확산하며 구성원들의 변화와 성장을 체계적으로 지원하고 있다.\n2025년, SKMS&Growth 조직의 신임임원으로 선임된 이송만 부사장은 “구성원 한 명, 한 명이 성장해야 회사도 성장할 수 있다”며 HRD의 중요성을 강조했다. 뉴스룸은 이 부사장을 만나 앞으로의 계획을 들어봤다.\n이 부사장은 27년간 SK그룹에서 다양한 커리어를 쌓았다. 이 부사장이 SK하이닉스에 합류한 것은 지난 2022년. 오랜 기간 SK그룹에서 인재 육성과 SKMS 확산 업무를 수행해 온 그는 “SK하이닉스의 구성원들에게서 더욱 특별한 점을 찾아볼 수 있었다”고 소감을 전했다.\n“우리 SK하이닉스 구성원들이 보여준 스스로 성장하고자 하는 ‘성장 의지와 욕구’는 매우 고무적이었습니다. 일반적으로 타 회사의 경우, 직무 육성 프로그램을 진행하면 전방위적인 홍보를 병행해야 참여도가 높아지기 마련인데요. SK하이닉스에서는 홍보가 없어도 수강 신청 서버가 마비될 정도로 폭발적인 관심을 받는 일이 많았습니다. 이러한 상황들을 보며, 우리 구성원들이 자신의 성장을 위해 VWBE와 SUPEX 추구를 실천하고 있다고 생각했습니다.”\n이 부사장은 신임임원으로 선임된 것에 관해 막중한 책임감을 느낀다며, 소감을 이어갔다.\n“기쁜 마음도 있습니다만, 우리 회사의 현재와 미래를 이끌어갈 전문가들을 육성해야 한다는 막중한 책임감과 사명감을 더 크게 느끼고 있습니다. 유독 뜨거운 우리 구성원들의 성장 의지와 욕구를 충족하고 지원해 드리기 위

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()
test_sentence = "아버지가방에들어가신다"
print(mecab.morphs(test_sentence))

['아버지', '가', '방', '에', '들어가', '신다']


In [4]:
#  LDA 모델링 (KoNLPy + Gensim)

mecab = Mecab()
texts = [[word for word in mecab.nouns(doc) if len(word) > 1] for doc in documents]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)
for topic in lda_model.print_topics(num_words=5):
    print(topic)



(0, '0.020*"기술" + 0.016*"하이닉스" + 0.014*"메모리" + 0.013*"구성원" + 0.013*"회사"')
(1, '0.026*"기술" + 0.025*"하이닉스" + 0.023*"메모리" + 0.020*"제품" + 0.017*"개발"')
(2, '0.026*"기술" + 0.025*"부사장" + 0.022*"하이닉스" + 0.021*"개발" + 0.013*"구성원"')


In [5]:
# BERTopic 모델링
from bertopic import BERTopic
import torch
from transformers import logging
logging.set_verbosity_error()  # transformers 경고 메시지 숨기기

# GPU 사용 가능한 경우 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BERTopic 모델 초기화 (더 가벼운 모델 사용)
topic_model = BERTopic(
    language="multilingual",
    embedding_model="paraphrase-multilingual-mpnet-base-v2"
)

# 문서에 대해 토픽 모델링 수행
topics, probs = topic_model.fit_transform(documents)

# 주요 토픽 5개 출력
top_topics = topic_model.get_topics()
for topic_num in list(top_topics.keys())[:5]:
    print(f"Topic {topic_num}: {top_topics[topic_num]}")


/home/arkwith/Dev/new_scraping/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'init_empty_weights' is not defined

In [ ]:
# 토픽 시각화
topic_model.visualize_topics()
topic_model.visualize_barchart(top_n_topics=5)
